In [14]:
# Import packages
import ast
import json
from enum import Enum
from pathlib import Path
import psycopg2
import ollama
import pandas as pd
from IPython.display import Image, Markdown, display
from tqdm import tqdm
import os
import urllib.parse
from sqlalchemy import create_engine

In [2]:
MODEL = "gpt-oss:20b"
TEMPERATURE = 0
DB_NAME=os.getenv('DB_NAME')
USERNAME=os.getenv('USERNAME')
PASSWORD=urllib.parse.quote(os.getenv('PASSWORD'))
HOSTNAME=os.getenv('HOSTNAME')
PORT=os.getenv('PORT')

In [3]:
# Creating conneciton to database
engine = create_engine(f'postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOSTNAME}/{DB_NAME}')

In [4]:
accounts_receivable = pd.read_sql("SELECT * FROM accounts_receivable", engine)
accounts_receivable

,invoice_number,date,customer_name,customer_number,amount,due_date,payment,payment_date,payment_id
0,1,2025-05-01,Planet Express,12038,50000.0,2025-05-02,None,None,None
1,2,2025-09-01,Mom's Friendly Robot Factory,12000,100000.0,2025-09-02,None,None,None
2,3,2025-10-01,Romanticorp,12990,73640.0,2025-10-02,None,None,None
3,4,2025-01-18,Hal Insitute for Criminally Insane Robots,12010,12500.0,2025-02-18,None,None,None
4,5,2025-01-21,Cookieville Minimum-Security Orphanarium,11900,10000.0,2025-02-21,None,None,None
5,6,2025-01-29,Panucci's Pizza,12933,1000.0,2025-02-28,None,None,None
6,7,2025-02-02,Planet Express,12038,12300.0,2025-02-03,None,None,None
7,8,2025-02-02,Romanticorp,12990,50000.0,2025-02-03,None,None,None
8,9,2025-03-02,Malfunctioning Eddie's Rocket-Car Emporium,12230,76000.0,2025-03-03,None,None,None
9,10,2025-05-02,Cookieville Minimum-Security Orphanarium,11900,80000.0,2025-05-02,None,None,None


In [5]:
payments= pd.read_sql("SELECT * FROM payments", engine)
payments

,transaction_id,payment_date,payment_amount,payment_reference
0,948347,2025-01-31,25000.0,12038 - 1 - We'll pay the rest later
1,34847,2025-01-20,100000.0,Customer Nr 12000 - Invoice Nr 2 - Mom's
2,29304,2025-01-02,12500.0,12010 - 4
3,3837459,2025-10-02,10000.0,11900 - 5
4,390576,2025-02-14,76000.0,12990 - Invoice Nr. 3
5,302947,2025-02-18,25000.0,12038 - 1 - Second payment
6,39506,2025-02-28,1000.0,12933 - 6 - Grazie
7,2394759,2025-01-03,76000.0,A little something for your troubles
8,390475,2025-03-13,80000.0,11900 -10


In [6]:
class ResponseFormat(Enum):
    JSON = "json_object"
    TEXT = "text"


def call_model(
    prompt: str, response_format: ResponseFormat = ResponseFormat.TEXT
) -> str:
    response = ollama.generate(
        model=MODEL,
        prompt=prompt,
        keep_alive="1h",
        format="" if response_format == ResponseFormat.TEXT else "json",
        options={"temperature": TEMPERATURE},
    )
    return response["response"]

In [7]:
%%time
task = f""" You have been given an accounts receivable invoice {accounts_receivable} and a received payment {payments}. Assess whether the payment relates to the invoice. 
            If there is no match, return an empty dictionary. If it does, UPDATE the invoice data with relevant payment information and return 
            the updated record as a dictionary with all invoice fields. Do not create any new columns for accounts receivables. Return the updated accounts receivables data as
            a dictionary with no other additional text."""
response = call_model(task)

CPU times: user 75.3 ms, sys: 489 ms, total: 564 ms
Wall time: 45min 24s


In [21]:
print(response)

{"1":{"invoice_number":1,"date":"2025-05-01","customer_name":"Planet Express","customer_number":12038,"amount":50000.0,"due_date":"2025-05-02","payment":50000.0,"payment_date":"2025-02-18","payment_id":302947},"2":{"invoice_number":2,"date":"2025-09-01","customer_name":"Mom's Friendly Robot Factory","customer_number":12000,"amount":100000.0,"due_date":"2025-09-02","payment":100000.0,"payment_date":"2025-01-20","payment_id":34847},"3":{"invoice_number":3,"date":"2025-10-01","customer_name":"Romanticorp","customer_number":12990,"amount":73640.0,"due_date":"2025-10-02","payment":12500.0,"payment_date":"2025-01-02","payment_id":29304},"4":{"invoice_number":4,"date":"2025-01-18","customer_name":"Hal Insitute for Criminally Insane Robots","customer_number":12010,"amount":12500.0,"due_date":"2025-02-18","payment":10000.0,"payment_date":"2025-10-02","payment_id":3837459},"5":{"invoice_number":5,"date":"2025-01-21","customer_name":"Cookieville Minimum-Security Orphanarium","customer_number":119

In [26]:
data = json.loads(response)

In [27]:
data

{'1': {'invoice_number': 1,
  'date': '2025-05-01',
  'customer_name': 'Planet Express',
  'customer_number': 12038,
  'amount': 50000.0,
  'due_date': '2025-05-02',
  'payment': 50000.0,
  'payment_date': '2025-02-18',
  'payment_id': 302947},
 '2': {'invoice_number': 2,
  'date': '2025-09-01',
  'customer_name': "Mom's Friendly Robot Factory",
  'customer_number': 12000,
  'amount': 100000.0,
  'due_date': '2025-09-02',
  'payment': 100000.0,
  'payment_date': '2025-01-20',
  'payment_id': 34847},
 '3': {'invoice_number': 3,
  'date': '2025-10-01',
  'customer_name': 'Romanticorp',
  'customer_number': 12990,
  'amount': 73640.0,
  'due_date': '2025-10-02',
  'payment': 12500.0,
  'payment_date': '2025-01-02',
  'payment_id': 29304},
 '4': {'invoice_number': 4,
  'date': '2025-01-18',
  'customer_name': 'Hal Insitute for Criminally Insane Robots',
  'customer_number': 12010,
  'amount': 12500.0,
  'due_date': '2025-02-18',
  'payment': 10000.0,
  'payment_date': '2025-10-02',
  'paym

In [45]:
df = pd.DataFrame()
for i in range(1,len(data)+1):
    ph_df = pd.json_normalize(data[f"{i}"])
    df = pd.concat([df, ph_df])
df

/var/folders/4s/q4zndhns13bdmf7q1_8p3crc0000gn/T/ipykernel_68564/472876252.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, ph_df])


,invoice_number,date,customer_name,customer_number,amount,due_date,payment,payment_date,payment_id
0,1,2025-05-01,Planet Express,12038,50000.0,2025-05-02,50000.0,2025-02-18,302947
0,2,2025-09-01,Mom's Friendly Robot Factory,12000,100000.0,2025-09-02,100000.0,2025-01-20,34847
0,3,2025-10-01,Romanticorp,12990,73640.0,2025-10-02,12500.0,2025-01-02,29304
0,4,2025-01-18,Hal Insitute for Criminally Insane Robots,12010,12500.0,2025-02-18,10000.0,2025-10-02,3837459
0,5,2025-01-21,Cookieville Minimum-Security Orphanarium,11900,10000.0,2025-02-21,76000.0,2025-02-14,390576
0,6,2025-01-29,Panucci's Pizza,12933,1000.0,2025-02-28,1000.0,2025-02-28,39506
0,7,2025-02-02,Planet Express,12038,12300.0,2025-02-03,NaN,None,None
0,8,2025-02-02,Romanticorp,12990,50000.0,2025-02-03,NaN,None,None
0,9,2025-03-02,Malfunctioning Eddie's Rocket-Car Emporium,12230,76000.0,2025-03-03,NaN,None,None
0,10,2025-05-02,Cookieville Minimum-Security Orphanarium,11900,80000.0,2025-05-02,80000.0,2025-03-13,390475
